### Prerequisites

You should have completed steps 1-3 of this tutorial before beginning this exercise.  The files required for this notebook are generated by those previous steps.

This notebook takes approximately 3 hours to run on an AWS `p3.8xlarge` instance. 

In [7]:
# # Optional: you can set what GPU you want to use in a notebook like this.  
# # Useful if you want to run concurrent experiments at the same time on different GPUs.
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [8]:
from pathlib import Path
import numpy as np
from seq2seq_utils import extract_encoder_model, load_encoder_inputs
from keras.layers import Input, Dense, BatchNormalization, Dropout, Lambda

from keras.models import load_model, Model
from seq2seq_utils import load_text_processor

#where you will save artifacts from this step
OUTPUT_PATH = Path('./data/code2emb/')
OUTPUT_PATH.mkdir(exist_ok=True)

# These are where the artifacts are stored from steps 2 and 3, respectively.
seq2seq_path = Path('./data/seq2seq/')
langemb_path = Path('./data/lang_model_emb/')

# set seeds
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

# Train Model That Maps Code To Sentence Embedding Space

In step 2, we trained a seq2seq model that can summarize function code using `(code, docstring)` pairs as the training data.  

In this step, we will fine tune the encoder from the seq2seq model to generate code embeddings in the docstring space by using `(code, docstring-embeddings)` as the training data.  Therefore, this notebook will go through the following steps:

1. Load the seq2seq model and extract the encoder (remember seq2seq models have an encoder and a decoder).
2. Freeze the weights of the encoder.
3. Add some dense layers on top of the encoder.
4. Train this new model supplying by supplying `(code, docstring-embeddings)` pairs.  We will call this model `code2emb_model`.
5. Unfreeze the entire model, and resume training.  This helps fine tune the model a little more towards this task.
6. Encode all of the code, including code that does not contain a docstring and save that into a search index for future use.  

### Load seq2seq model from Step 2 and extract the encoder

First load the seq2seq model from Step2, then extract the encoder (we do not need the decoder).

In [9]:
# load the pre-processed data for the encoder (we don't care about the decoder in this step)
encoder_input_data, doc_length = load_encoder_inputs(seq2seq_path/'py_t_code_vecs_v2.npy')
seq2seq_Model = load_model(seq2seq_path/'code_summary_seq2seq_model.h5')

Shape of encoder input: (244125, 55)


In [10]:
# Extract Encoder from seq2seq model
encoder_model = extract_encoder_model(seq2seq_Model)
# Get a summary of the encoder and its layers
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder-Input (InputLayer)   (None, 55)                0         
_________________________________________________________________
Body-Word-Embedding (Embeddi (None, 55, 800)           16001600  
_________________________________________________________________
Encoder-Batchnorm-1 (BatchNo (None, 55, 800)           3200      
_________________________________________________________________
Encoder-Last-GRU (GRU)       [(None, 1000), (None, 100 5403000   
Total params: 21,407,800
Trainable params: 21,406,200
Non-trainable params: 1,600
_________________________________________________________________


Freeze the encoder

In [11]:
# Freeze Encoder Model
for l in encoder_model.layers:
    l.trainable = False
    print(l, l.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f305369bb00> False
<keras.layers.embeddings.Embedding object at 0x7f305369beb8> False
<keras.layers.normalization.BatchNormalization object at 0x7f305369b9e8> False
<keras.layers.recurrent.GRU object at 0x7f305369ba58> False


### Load Docstring Embeddings From From Step 3

The target for our `code2emb` model will be docstring-embeddings instead of docstrings.  Therefore, we will use the embeddings for docstrings that we computed in step 3.  For this tutorial, we will use the average over all hidden states, which is saved in the file `avg_emb_dim500_v2.npy`.

Note that in our experiments, a concatenation of the average, max, and last hidden state worked better than using the average alone.  However, in the interest of simplicity we demonstrate just using the average hidden state.  We leave it as an exercise to the reader to experiment with other approaches. 

In [13]:
# Load Fitlam Embeddings
fastailm_emb = np.load(langemb_path/'trn_avg_emb_dim500_v2.npy')
print(encoder_input_data.shape[0], fastailm_emb.shape[0])
# check that the encoder inputs have the same number of rows as the docstring embeddings
assert encoder_input_data.shape[0] == fastailm_emb.shape[0]

fastailm_emb.shape

244125 244125


(244125, 500)

### Construct `code2emb` Model Architecture

The `code2emb` model is the encoder from the seq2seq model with some dense layers added on top.  The output of the last dense layer of this model needs to match the dimensionality of the docstring embedding, which is 500 in this case.

In [14]:
#### Encoder Model ####
encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')
enc_out = encoder_model(encoder_inputs)

# first dense layer with batch norm
x = Dense(500, activation='relu')(enc_out)
x = BatchNormalization(name='bn-1')(x)
out = Dense(500)(x)
code2emb_model = Model([encoder_inputs], out)

In [15]:
code2emb_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder-Input (InputLayer)   (None, 55)                0         
_________________________________________________________________
Encoder-Model (Model)        (None, 1000)              21407800  
_________________________________________________________________
dense_1 (Dense)              (None, 500)               500500    
_________________________________________________________________
bn-1 (BatchNormalization)    (None, 500)               2000      
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
Total params: 22,160,800
Trainable params: 752,000
Non-trainable params: 21,408,800
_________________________________________________________________


### Train the `code2emb` Model

The model we are training is relatively simple - with two dense layers on top of the pre-trained encoder.  We are leaving the encoder frozen at first, then will unfreeze the encoder in a later step.

In [16]:
from keras.callbacks import Callback
import json
from pathlib import Path
from seq2seq_utils import load_text_processor
from general_utils import create_nmslib_search_index
from lang_model_utils import load_lm_vocab, Query2Emb
import torch

seq2seq_path = Path('./data/seq2seq/')
threshold =1

class EvaluateCode2Emd(Callback):
    def evaluate(self):
        with open('validation_set_js.json', 'r') as f:
#             [query, code, isPositive]
            valid_pairs = json.load(f)
        all = 0
        find = 0
        num_encoder_tokens, enc_pp = load_text_processor(seq2seq_path/'py_code_proc_v2.dpkl')
        lang_model = torch.load('./data/lang_model/lang_model_cpu_v2.torch', 
                            map_location=lambda storage, loc: storage)
        vocab = load_lm_vocab('./data/lang_model/vocab_v2.cls')
        q2emb = Query2Emb(lang_model = lang_model.cpu(), vocab = vocab).emb_mean
        i=0
        for pair in valid_pairs:
            lang_model.reset()
    #       code 2 emb
            encinp = enc_pp.transform([pair['result']['code']])
            nodoc_vecs = self.model.predict(encinp, batch_size=1)
            search_index = create_nmslib_search_index(nodoc_vecs)
    #       query 2 emb  
            query = q2emb(pair['query'])
#             search
            idxs, dists = search_index.knnQuery(query, k=1)
            if dists[0] < threshold:
                find += 1
            all += 1
        assert all > 0
        return find/all
    
    def on_epoch_end(self, epoch, logs):
        coverage = self.evaluate()
        print('epoch:', epoch, 'coverage:', coverage)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [18]:
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras import optimizers

code2emb_model.compile(optimizer=optimizers.Nadam(lr=0.002), loss='cosine_proximity')
script_name_base = 'code2emb_model_'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)
evaluater = EvaluateCode2Emd()
batch_size = 2000
epochs = 16
history = code2emb_model.fit([encoder_input_data], fastailm_emb,
          batch_size=batch_size,
          epochs=epochs)

Epoch 1/16
244125/244125 [==============================] - 100s 409us/step - loss: -0.7435
Epoch 2/16
244125/244125 [==============================] - 99s 406us/step - loss: -0.7532
Epoch 3/16
244125/244125 [==============================] - 99s 407us/step - loss: -0.7564
Epoch 4/16
244125/244125 [==============================] - 100s 408us/step - loss: -0.7584
Epoch 5/16
244125/244125 [==============================] - 100s 409us/step - loss: -0.7606
Epoch 6/16
244125/244125 [==============================] - 100s 410us/step - loss: -0.7621
Epoch 7/16
244125/244125 [==============================] - 100s 411us/step - loss: -0.7632
Epoch 8/16
244125/244125 [==============================] - 100s 411us/step - loss: -0.7643
Epoch 9/16
244125/244125 [==============================] - 100s 411us/step - loss: -0.7648
Epoch 10/16
244125/244125 [==============================] - 101s 412us/step - loss: -0.7656
Epoch 11/16
244125/244125 [==============================] - 100s 412us/step - lo

`.7453`

### Unfreeze all Layers of Model and Resume Training

In the previous step, we left the encoder frozen.  Now that the dense layers are trained, we will unfreeze the entire model and let it train some more.  This will hopefully allow this model to specialize on this task a bit more.

In [19]:
for l in code2emb_model.layers:
    l.trainable = True
    print(l, l.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f305ea3d5f8> True
<keras.engine.training.Model object at 0x7f305376a400> True
<keras.layers.core.Dense object at 0x7f305ea3d710> True
<keras.layers.normalization.BatchNormalization object at 0x7f305ea3da90> True
<keras.layers.core.Dense object at 0x7f3055949940> True


In [28]:
code2emb_model.compile(optimizer=optimizers.Nadam(lr=0.0001), loss='cosine_proximity')
script_name_base = 'code2emb_model_unfreeze_'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)
evaluater = EvaluateCode2Emd()
batch_size = 2000
epochs = 20
history = code2emb_model.fit([encoder_input_data], fastailm_emb,
          batch_size=batch_size,
          epochs=epochs,
          initial_epoch=16,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

Train on 214830 samples, validate on 29295 samples
Epoch 17/20
214830/214830 [==============================] - 98s 458us/step - loss: -0.7712 - val_loss: -0.7806
Epoch 18/20
214830/214830 [==============================] - 97s 451us/step - loss: -0.7713 - val_loss: -0.7807
Epoch 19/20
214830/214830 [==============================] - 97s 454us/step - loss: -0.7713 - val_loss: -0.7808
Epoch 20/20
214830/214830 [==============================] - 98s 456us/step - loss: -0.7713 - val_loss: -0.7807


### Save `code2emb` model

In [29]:
code2emb_model.save(OUTPUT_PATH/'code2emb_model.hdf5')

This file has been cached and is also available for download here:

`code2emb_model.hdf5`:https://storage.googleapis.com/kubeflow-examples/code_search/data/code2emb/code2emb_model.hdf5

# Vectorize all of the code without docstrings

We want to vectorize all of the code without docstrings so we can test the efficacy of the search on the code that was never seen by the model. 

In [30]:
from keras.models import load_model
from pathlib import Path
import numpy as np
from seq2seq_utils import load_text_processor
code2emb_path = Path('./data/code2emb/')
seq2seq_path = Path('./data/seq2seq/')
data_path = Path('./data/processed_data/')

In [31]:
code2emb_model = load_model(code2emb_path/'code2emb_model.hdf5')
num_encoder_tokens, enc_pp = load_text_processor(seq2seq_path/'py_code_proc_v2.dpkl')

with open(data_path/'without_docstrings.function', 'r') as f:
    no_docstring_funcs = f.readlines()

Size of vocabulary for data/seq2seq/py_code_proc_v2.dpkl: 20,002


In [ ]:
with open(data_path/'train.function', 'r') as f:
    train_funcs = f.readlines()
with open(data_path/'valid.function', 'r') as f:
    valid_funcs = f.readlines()
with open(data_path/'test.function', 'r') as f:
    test_funcs = f.readlines()

In [ ]:
total_funcs = no_docstring_funcs + train_funcs + valid_funcs + test_funcs

### Pre-process code without docstrings for input into `code2emb` model

We use the same transformer we used to train the original model.

In [32]:
# tokenized functions that did not contain docstrigns
no_docstring_funcs[:5]

['function _interopRequireDefault obj return obj obj __esModule obj default obj\n',
 'function t var walt n import extern Add from env n type Add i32 i32 0 i32 n function add x i32 y i32 1 i32 n return x y n n n export function test i32 n const x i32 add 2 n const y i32 extern 1 0 n return x y n return WebAssembly instantiate 0 _ compile walt buffer env extern function extern k i return k i\n',
 'function mod t is mod instance exports test 4\n',
 'function t var walt n shadowing variables should be OK n export function test i32 n Definiong an identifier which is a function name n const test i32 42 n n The function pointer parser should defer to available scopes first and n not re map this identifier to a pointer n return test n n return WebAssembly instantiate 0 _ compile walt buffer then function mod t is mod instance exports test 42\n',
 'function t var src n For pointers n const table Table element anyfunc initial 10 max 10 n For object operations n const memory Memory initial 1 n n

In [33]:
encinp = enc_pp.transform_parallel(total_funcs)
np.save(code2emb_path/'total_funcs.npy', encinp)

### Extract code vectors

In [34]:
from keras.models import load_model
from pathlib import Path
import numpy as np
code2emb_path = Path('./data/code2emb/')
encinp = np.load(code2emb_path/'total_funcs.npy')
code2emb_model = load_model(code2emb_path/'code2emb_model.hdf5')

Use the `code2emb` model to map the code into the same vector space as natural language 

In [36]:
vecs = code2emb_model.predict(encinp, batch_size=2000)

In [37]:
# make sure the number of output rows equal the number of input rows
assert nodoc_vecs.shape[0] == encinp.shape[0]

Save the vectorized code

In [38]:
np.save(code2emb_path/'vecs.npy', nodoc_vecs)

# Cached Files

You can find the files that were created in this notebook below.  **Please note that if you use one of these files, you should proceed with extreme caution.**  We recommend that if you are skipping a step, you should use *all* the cached files because only using only some files could result in discrepencies between your models or data and our pre-computed results.

1. `code2emb_model.hdf5`:https://storage.googleapis.com/kubeflow-examples/code_search/data/code2emb/code2emb_model.hdf5
2. `nodoc_encinp.npy`:https://storage.googleapis.com/kubeflow-examples/code_search/data/code2emb/nodoc_encinp.npy
3. `nodoc_vecs.npy`:https://storage.googleapis.com/kubeflow-examples/code_search/data/code2emb/nodoc_vecs.npy